In [91]:
from datetime import datetime
import pandas as pd
import locale
import warnings
import os

locale.setlocale(locale.LC_TIME, 'es_ES.UTF-8')
warnings.filterwarnings("ignore")

In [92]:
def get_file_name() -> list[str]:
    file_list = []
    for file in os.listdir():
        if file.endswith('.xlsx') and 'Base Pagos' in file:
            file_list.append(file)
    return file_list

def get_last_date(dates_list: list[datetime]) -> datetime:
    last_date = dates_list[0]
    for date in dates_list:
        if date > last_date:
            last_date = date
    return last_date

def get_date(file_list: list[str]) -> str:
    dates_list = []
    for file in file_list:
        fecha_list =  file.split(' ')[-1].split('.')[:3]
        dates_list.append(datetime.strptime(fecha_list[0] + '.' + fecha_list[1] + '.' + fecha_list[2], '%d.%m.%Y'))
    return get_last_date(dates_list).strftime('%d.%m.%Y')

fecha = get_date(get_file_name())
fecha

'27.08.2024'

In [93]:
def validar_o_crear_carpeta() -> str:
    fecha_hoy = datetime.today().strftime('%Y.%m.%d')
    
    carpeta_reporte = 'reporte'
    
    carpeta_hoy = os.path.join(carpeta_reporte, fecha_hoy)
    
    if not os.path.exists(carpeta_hoy):
        os.makedirs(carpeta_hoy)
        print(f'Carpeta creada: {carpeta_hoy}')
    else:
        print(f'La carpeta ya existe: {carpeta_hoy}')
    
    return carpeta_hoy

file_path = validar_o_crear_carpeta()
file_path

La carpeta ya existe: reporte\2024.08.28


'reporte\\2024.08.28'

In [94]:
base_pagos_path = 'Base Pagos ' + fecha + '.xlsx'
asignacion_path = 'base_asignacion_20240813.xlsx'

print(base_pagos_path)
print(asignacion_path)

monoproducto = file_path + '/MONOPRODUCTO ' + fecha + '.xlsx'
multiproducto = file_path + '/MULTIPRODUCTO ' + fecha + '.xlsx'
reactiva = file_path + '/REACTIVA ' + fecha + '.xlsx'
no_enviados = file_path + '/NO ENVIADOS ' + fecha + '.xlsx'

mono_path = os.path.abspath(monoproducto)
multi_path = os.path.abspath(multiproducto)
react_path = os.path.abspath(reactiva)
no_enviados_path = os.path.abspath(no_enviados)

print(mono_path)
print(multi_path)
print(react_path)
print(no_enviados_path)

Base Pagos 27.08.2024.xlsx
base_asignacion_20240813.xlsx
c:\Users\miria\Documents\Mirko\BBVA\GitHub\base-pagos\reporte\2024.08.28\MONOPRODUCTO 27.08.2024.xlsx
c:\Users\miria\Documents\Mirko\BBVA\GitHub\base-pagos\reporte\2024.08.28\MULTIPRODUCTO 27.08.2024.xlsx
c:\Users\miria\Documents\Mirko\BBVA\GitHub\base-pagos\reporte\2024.08.28\REACTIVA 27.08.2024.xlsx
c:\Users\miria\Documents\Mirko\BBVA\GitHub\base-pagos\reporte\2024.08.28\NO ENVIADOS 27.08.2024.xlsx


In [95]:
#################### BASE DE PAGOS ####################

In [96]:
df_base = pd.read_excel(base_pagos_path)
print(df_base.shape)
print(df_base.dtypes)
df_base.head(5)

(435, 9)
Entidad         int64
FECHA          object
CODCEN          int64
CLAVSERV        int64
CENTROPAGO      int64
IMPORTE       float64
MONEDA         object
NOMBRE         object
ESTADO         object
dtype: object


,Entidad,FECHA,CODCEN,CLAVSERV,CENTROPAGO,IMPORTE,MONEDA,NOMBRE,ESTADO
0,11,2024-08-27,23307614,248,849,500.0,PEN,-VICTOR RAUL VELASQUEZ CENTENO,P
1,11,2024-08-27,28256566,248,843,2000.0,PEN,-MARLON ERNESTO ZAVALA ALVAREZ.,P
2,11,2024-08-27,29572732,248,14,900.0,PEN,ABEL QUISPE SULLCA,P
3,11,2024-08-27,27204708,248,195,186.0,PEN,ABRAHAM ALBERTO SEMINARIO SAN MARTIN,P
4,11,2024-08-27,27204708,248,195,1178.0,PEN,ABRAHAM ALBERTO SEMINARIO SAN MARTIN.,P


In [97]:
fecha_formateada = pd.to_datetime('today').strftime('%d-%b')
fecha_formateada = fecha_formateada[:3] + fecha_formateada[3:].capitalize()

df_base['FECHA_ENVIO'] = fecha_formateada.replace('.', '')
df_base['ID_RESPONSABLE'] = 'MIV'

cols_base = ['Entidad', 'FECHA', 'CODCEN', 'CLAVSERV', 'CENTROPAGO', 'IMPORTE', 'MONEDA', 'NOMBRE', 'ESTADO', 'FECHA_ENVIO', 'ID_RESPONSABLE']
df_base = df_base[cols_base]
df_base = df_base.rename(columns={'CODCEN': 'CC'})

df_base['CC'] = df_base['CC'].astype('Int64').astype(str).str.zfill(8)
df_base['CC'] = df_base['CC'].str.replace(' ', '').str.replace(r'\D', '', regex=True).str[-8:]
df_base['CC'] = df_base['CC'].str.zfill(8)

df_base['CLAVSERV'] = df_base['CLAVSERV'].astype(str).str.zfill(5)

print(df_base.shape)
print(df_base.dtypes)
df_base.head(5)

(435, 11)
Entidad             int64
FECHA              object
CC                 object
CLAVSERV           object
CENTROPAGO          int64
IMPORTE           float64
MONEDA             object
NOMBRE             object
ESTADO             object
FECHA_ENVIO        object
ID_RESPONSABLE     object
dtype: object


,Entidad,FECHA,CC,CLAVSERV,CENTROPAGO,IMPORTE,MONEDA,NOMBRE,ESTADO,FECHA_ENVIO,ID_RESPONSABLE
0,11,2024-08-27,23307614,00248,849,500.0,PEN,-VICTOR RAUL VELASQUEZ CENTENO,P,28-Ago,MIV
1,11,2024-08-27,28256566,00248,843,2000.0,PEN,-MARLON ERNESTO ZAVALA ALVAREZ.,P,28-Ago,MIV
2,11,2024-08-27,29572732,00248,14,900.0,PEN,ABEL QUISPE SULLCA,P,28-Ago,MIV
3,11,2024-08-27,27204708,00248,195,186.0,PEN,ABRAHAM ALBERTO SEMINARIO SAN MARTIN,P,28-Ago,MIV
4,11,2024-08-27,27204708,00248,195,1178.0,PEN,ABRAHAM ALBERTO SEMINARIO SAN MARTIN.,P,28-Ago,MIV


In [98]:
df_base.isnull().sum()

Entidad           0
FECHA             0
CC                0
CLAVSERV          0
CENTROPAGO        0
IMPORTE           0
MONEDA            0
NOMBRE            0
ESTADO            0
FECHA_ENVIO       0
ID_RESPONSABLE    0
dtype: int64

In [99]:
#################### ASIGNACION ####################

In [100]:
if not os.path.exists('temp_asignacion_'+ fecha +'.txt'):
    df_asignacion = pd.read_excel(asignacion_path)
    with open('temp_asignacion_'+ fecha +'.txt', 'w') as file:
        file.close()

print(df_asignacion.shape)
print(df_asignacion.dtypes)
df_asignacion.head(5)

(242511, 40)
codigo                        int64
contrato                      int64
cont_18                       int64
nombre                       object
clave                        object
producto                     object
DiasVenc                      int64
moneda                       object
capital                     float64
liqtotal                    float64
agencia                      object
Camp                        float64
Gestor                       object
numdoc                       object
oficina                      object
territorio                   object
Tramo                        object
codoficina                    int64
CAPITAL_SOLES               float64
IMP_CAMPAÑA                 float64
TIPO_FONDO                   object
FLAG_HONRADO                  int64
TIPO_CARTERA                 object
DESC_TRAMO                   object
Hibrido                      object
ESTUDIO ABOGADO              object
FECHA_MORA                   object
ID_CASTIGO     

,codigo,contrato,cont_18,nombre,clave,producto,DiasVenc,moneda,capital,liqtotal,...,ID_CONDONACION MORA_TEMP,ID_FRACCIONADO,ID_KOBSA_P.CANCELACION,ID_REPORTE_CANC,ID_CARTERA,ID_FLAG,OBSERVACION,AGENCIA CORRECTA,PAG LEGAL,PAG. INTER
0,30739481,110010629600055161,1100109600055161,DINA CHOQUE FERNANDEZ,BC,PRÉSTAMOS PYMES,257,PEN,76663.97,108418.69,...,-,-,-,-,STOCK,2,PAGO LEGAL AGOSTO // SE CORRIGIO,MORNESE MORA,1,-
1,30739481,110010619600057679,1100109600057679,DINA CHOQUE FERNANDEZ,BC,PRÉSTAMOS PYMES,268,PEN,177075.30,256680.65,...,-,-,-,-,STOCK,2,PAGO LEGAL AGOSTO // SE CORRIGIO,MORNESE MORA,1,-
2,21874900,110050295000360222,1100505000360222,JULIA ANGELICA SOTO COTRINA,BC,TARJETAS PYMES,307,PEN,23986.04,36966.81,...,-,-,-,-,STOCK,4,PAGO INTERBANCARIO // SE CORRIGIO,ASESCOM RJ,-,1
3,30293392,110130235002987960,1101305002987960,INNOVA FASHION GROUPSAC,BC,TARJETAS PYMES,215,PEN,142100.58,200698.07,...,-,-,-,-,STOCK,1,PAGO INTERBANCARIO // SE CORRIGIO,CLASA MORA,-,1
4,20687681,110174079600365104,1101749600365104,ELIZABETH HAYDEE CORDOVA AMAYA,BC,PRÉSTAMOS PYMES,356,PEN,200151.61,309435.56,...,-,1,-,-,STOCK,1,MARCA FRACCIONADO RETORNAR AGENCIA // SE CORRIGIO,RodriguezAngobaldo,-,-


In [101]:
#################### BACKUPS ####################

In [102]:
df_base_test = df_base.copy()
print('Base', df_base_test.shape)

df_asignacion_test = df_asignacion.copy()
print('Asignacion', df_asignacion_test.shape)

Base (435, 11)
Asignacion (242511, 40)


In [103]:
base_count = df_base_test.shape[0]
base_count

435

In [104]:
df_asignacion_test['codigo'] = df_asignacion_test['codigo'].astype('Int64').astype(str).str.zfill(8)

cols_asignacion = ['codigo', 'ID_FLAG', 'cont_18', 'nombre', 'TIPO_CARTERA', 'TIPO_FONDO', 'clave', 'AGENCIA CORRECTA', 'ID_VTA.']
df_base_test = df_base_test.merge(df_asignacion_test[cols_asignacion], left_on='CC', right_on='codigo', how='left')
df_base_test = df_base_test.drop(columns=['codigo'])
df_base_test = df_base_test.rename(columns={'Entidad':'ENTIDAD', 'ID_FLAG': 'FLAG', 'cont_18': 'CONTRATO', 'nombre': 'NOMBRE_CLIENTE', 'clave': 'CARTERA', 'AGENCIA CORRECTA': 'AGENCIA', 'ID_VTA.': 'ID_VTA'})

df_base_test.drop_duplicates(subset=['CC', 'IMPORTE', 'MONEDA', 'NOMBRE'], keep='first', inplace=True)

df_base_test['TIPO_CARTERA'] = df_base_test['TIPO_CARTERA'].fillna('NULL')
df_base_test['TIPO_FONDO'] = df_base_test['TIPO_FONDO'].fillna('NULL')

df_base_test['FLAG'] = df_base_test['FLAG'].astype('Int64')
df_base_test['CONTRATO'] = df_base_test['CONTRATO'].apply(lambda x: str(int(x)).zfill(18) if pd.notna(x) else x)
df_base_test['ENTIDAD'] = df_base_test['ENTIDAD'].astype('Int64').astype(str).str.zfill(4)

cols_base_test = ['ENTIDAD', 'FECHA', 'CC', 'CLAVSERV', 'CENTROPAGO', 'IMPORTE', 'MONEDA', 'NOMBRE', 'ESTADO', 'FLAG', 'CONTRATO', 'NOMBRE_CLIENTE', 'FECHA_ENVIO', 'ID_RESPONSABLE', 'TIPO_CARTERA', 'TIPO_FONDO', 'CARTERA', 'AGENCIA', 'ID_VTA']
df_base_test = df_base_test[cols_base_test]

print(df_base_test.shape)
print(df_base_test.dtypes)
df_base_test.head(5)

(435, 19)
ENTIDAD            object
FECHA              object
CC                 object
CLAVSERV           object
CENTROPAGO          int64
IMPORTE           float64
MONEDA             object
NOMBRE             object
ESTADO             object
FLAG                Int64
CONTRATO           object
NOMBRE_CLIENTE     object
FECHA_ENVIO        object
ID_RESPONSABLE     object
TIPO_CARTERA       object
TIPO_FONDO         object
CARTERA            object
AGENCIA            object
ID_VTA             object
dtype: object


,ENTIDAD,FECHA,CC,CLAVSERV,CENTROPAGO,IMPORTE,MONEDA,NOMBRE,ESTADO,FLAG,CONTRATO,NOMBRE_CLIENTE,FECHA_ENVIO,ID_RESPONSABLE,TIPO_CARTERA,TIPO_FONDO,CARTERA,AGENCIA,ID_VTA
0,0011,2024-08-27,23307614,00248,849,500.0,PEN,-VICTOR RAUL VELASQUEZ CENTENO,P,1,001101379600154101,VICTOR RAUL VELASQUEZ CENTENO,28-Ago,MIV,NULL,NULL,CDT,SIN AGENCIA,-
1,0011,2024-08-27,28256566,00248,843,2000.0,PEN,-MARLON ERNESTO ZAVALA ALVAREZ.,P,1,001109005000817850,MARLON ERNESTO ZAVALA ALVAREZ,28-Ago,MIV,UNSECURED,NULL,EXJ,MORNESE MORA,-
2,0011,2024-08-27,29572732,00248,14,900.0,PEN,ABEL QUISPE SULLCA,P,1,001100145000366666,ABEL QUISPE SULLCA,28-Ago,MIV,UNSECURED,NULL,EXJ,MORNESE MORA,-
3,0011,2024-08-27,27204708,00248,195,186.0,PEN,ABRAHAM ALBERTO SEMINARIO SAN MARTIN,P,2,001101955001801884,ABRAHAM ALBERTO SEMINARIO SAN MARTIN,28-Ago,MIV,UNSECURED,NULL,KSTBC,MORNESE MORA,-
5,0011,2024-08-27,27204708,00248,195,1178.0,PEN,ABRAHAM ALBERTO SEMINARIO SAN MARTIN.,P,2,001101955001801884,ABRAHAM ALBERTO SEMINARIO SAN MARTIN,28-Ago,MIV,UNSECURED,NULL,KSTBC,MORNESE MORA,-


In [105]:
if base_count == df_base_test.shape[0]:
    print('REGISTROS OK')
elif base_count > df_base_test.shape[0]:
    print('Se han perdido registros')
else:
    print('Se han duplicado registros: ', df_base_test.shape[0] - base_count)

REGISTROS OK


In [106]:
df_base_test.shape

(435, 19)

In [107]:
df_base_no_flag = df_base_test[df_base_test['FLAG'].isnull()]
df_base_no_flag.shape

(21, 19)

In [108]:
no_flag_count = df_base_no_flag.shape[0]
no_flag_count

21

In [109]:
flag_count = base_count - no_flag_count
flag_count

414

In [110]:
#################### MONOPRODUCTO ####################

In [111]:
df_mono = df_base_test[df_base_test['FLAG'] == 1]
print(df_mono.shape)
df_mono.head(5)

(324, 19)


,ENTIDAD,FECHA,CC,CLAVSERV,CENTROPAGO,IMPORTE,MONEDA,NOMBRE,ESTADO,FLAG,CONTRATO,NOMBRE_CLIENTE,FECHA_ENVIO,ID_RESPONSABLE,TIPO_CARTERA,TIPO_FONDO,CARTERA,AGENCIA,ID_VTA
0,0011,2024-08-27,23307614,00248,849,500.0,PEN,-VICTOR RAUL VELASQUEZ CENTENO,P,1,001101379600154101,VICTOR RAUL VELASQUEZ CENTENO,28-Ago,MIV,NULL,NULL,CDT,SIN AGENCIA,-
1,0011,2024-08-27,28256566,00248,843,2000.0,PEN,-MARLON ERNESTO ZAVALA ALVAREZ.,P,1,001109005000817850,MARLON ERNESTO ZAVALA ALVAREZ,28-Ago,MIV,UNSECURED,NULL,EXJ,MORNESE MORA,-
2,0011,2024-08-27,29572732,00248,14,900.0,PEN,ABEL QUISPE SULLCA,P,1,001100145000366666,ABEL QUISPE SULLCA,28-Ago,MIV,UNSECURED,NULL,EXJ,MORNESE MORA,-
7,0011,2024-08-27,27393417,00248,304,700.0,PEN,ABRIL MIHANU HOYOS PANDURO,P,1,001103175001197105,ABRIL MIHANU HOYOS PANDURO,28-Ago,MIV,UNSECURED,NULL,EXJ,MORNESE MORA,-
8,0011,2024-08-27,30106214,00248,234,13500.0,PEN,ACEROS Y VIDRIERIA DAKI SAC,P,1,001103239600179989,ACEROS Y VIDRIERIA DAKI SAC,28-Ago,MIV,UNSECURED,NULL,KSTBC,ASESCOM RJ,-


In [112]:
mono_count = df_mono.shape[0]
mono_count

324

In [113]:
df_mono['TIPO_CARTERA'].value_counts()

TIPO_CARTERA
UNSECURED    309
SECURED       14
NULL           1
Name: count, dtype: int64

In [114]:
df_mono['TIPO_FONDO'].value_counts()

TIPO_FONDO
NULL        316
REACTIVA      8
Name: count, dtype: int64

In [115]:
df_mono[['TIPO_CARTERA', 'TIPO_FONDO']].value_counts()

TIPO_CARTERA  TIPO_FONDO
UNSECURED     NULL          303
SECURED       NULL           12
UNSECURED     REACTIVA        6
SECURED       REACTIVA        2
NULL          NULL            1
Name: count, dtype: int64

In [116]:
#################### MONO REACTIVA ####################

In [117]:
df_reactiva = df_mono[(df_mono['TIPO_CARTERA'] == 'UNSECURED') & (df_mono['TIPO_FONDO'] == 'REACTIVA')]
df_reactiva.drop(columns=['ENTIDAD', 'CENTROPAGO', 'ID_VTA'], inplace=True)
print(df_reactiva.shape)
df_reactiva.head(5)

(6, 16)


,FECHA,CC,CLAVSERV,IMPORTE,MONEDA,NOMBRE,ESTADO,FLAG,CONTRATO,NOMBRE_CLIENTE,FECHA_ENVIO,ID_RESPONSABLE,TIPO_CARTERA,TIPO_FONDO,CARTERA,AGENCIA
84,2024-08-27,26404201,00248,2000.00,PEN,"COINGEMA INDUSTRIAL SAC,,,",P,1,001101419600146593,COINGEMA INDUSTRIALSAC,28-Ago,MIV,UNSECURED,REACTIVA,BC,CLASA MORA
149,2024-08-27,27692222,00248,1000.00,PEN,EMPAISLA EIRL,P,1,001101279600250750,EMPAISLA EIRL,28-Ago,MIV,UNSECURED,REACTIVA,BC,ASESCOM RJ
162,2024-08-27,24860259,00248,5000.00,PEN,FIBRAART PERU SAC,P,1,001101399600235560,FIBRAART PERU SAC,28-Ago,MIV,UNSECURED,REACTIVA,KSTBC,MORNESE MORA
194,2024-08-27,25448033,00248,10268.05,PEN,HUGO. MARCIAL AGUILAR. LIMA,P,1,001102209600865254,HUGO MARCIAL AGUILAR LIMA,28-Ago,MIV,UNSECURED,REACTIVA,EXJ,ASESCOM RJ
349,2024-08-27,22250963,00248,4000.00,PEN,MARIZA MERY MALLMA LIMACO 27/08/2024,P,1,001102359601000437,MARIZA MERY MALLMA LIMACO,28-Ago,MIV,UNSECURED,REACTIVA,BC,CLASA MORA


In [118]:
reactiva_count = df_reactiva.shape[0]
reactiva_count

6

In [119]:
df_reactiva.to_excel(reactiva, index=False)

In [120]:
#################### FIN REACTIVA ####################

In [121]:
df_no_mono = df_mono[(df_mono['TIPO_CARTERA'] != 'UNSECURED')]
no_mono_count = df_no_mono.shape[0]
no_mono_count

15

In [122]:
df_mono = df_mono[df_mono['TIPO_CARTERA'] == 'UNSECURED']
df_mono = df_mono[df_mono['TIPO_FONDO'] == 'NULL']
print(df_mono.shape)

(303, 19)


In [123]:
df_mono['CONTRATO'] = df_mono['CONTRATO'].astype(str).str.zfill(18)

In [124]:
df_mono.drop(columns=['ENTIDAD', 'CENTROPAGO', 'ID_VTA'], inplace=True)
print(df_mono.shape)
df_mono.head(5)

(303, 16)


,FECHA,CC,CLAVSERV,IMPORTE,MONEDA,NOMBRE,ESTADO,FLAG,CONTRATO,NOMBRE_CLIENTE,FECHA_ENVIO,ID_RESPONSABLE,TIPO_CARTERA,TIPO_FONDO,CARTERA,AGENCIA
1,2024-08-27,28256566,00248,2000.0,PEN,-MARLON ERNESTO ZAVALA ALVAREZ.,P,1,001109005000817850,MARLON ERNESTO ZAVALA ALVAREZ,28-Ago,MIV,UNSECURED,NULL,EXJ,MORNESE MORA
2,2024-08-27,29572732,00248,900.0,PEN,ABEL QUISPE SULLCA,P,1,001100145000366666,ABEL QUISPE SULLCA,28-Ago,MIV,UNSECURED,NULL,EXJ,MORNESE MORA
7,2024-08-27,27393417,00248,700.0,PEN,ABRIL MIHANU HOYOS PANDURO,P,1,001103175001197105,ABRIL MIHANU HOYOS PANDURO,28-Ago,MIV,UNSECURED,NULL,EXJ,MORNESE MORA
8,2024-08-27,30106214,00248,13500.0,PEN,ACEROS Y VIDRIERIA DAKI SAC,P,1,001103239600179989,ACEROS Y VIDRIERIA DAKI SAC,28-Ago,MIV,UNSECURED,NULL,KSTBC,ASESCOM RJ
9,2024-08-27,21557284,00248,2717.0,PEN,ADELA CARIDAD RUIZ ARCE,P,1,001103019601222187,ADELA CARIDAD RUIZ ARCE,28-Ago,MIV,UNSECURED,NULL,KSTBC,ASESCOM RJ


In [125]:
df_mono.to_excel(monoproducto, index=False)

In [126]:
mono_count_final = df_mono.shape[0]
mono_count_final

303

In [127]:
if mono_count == mono_count_final + reactiva_count + no_mono_count:
    print('MONOPRODUCTO OK')
else:
    print('MONOPRODUCTO ERROR')

MONOPRODUCTO OK


In [128]:
#################### FIN MONOPRODUCTO ####################

In [129]:
#################### MULTIPRODUCTO ####################

In [130]:
df_multi = df_base_test[df_base_test['FLAG'] > 1].copy()
print(df_multi.shape)
df_multi.head(5)

(90, 19)


,ENTIDAD,FECHA,CC,CLAVSERV,CENTROPAGO,IMPORTE,MONEDA,NOMBRE,ESTADO,FLAG,CONTRATO,NOMBRE_CLIENTE,FECHA_ENVIO,ID_RESPONSABLE,TIPO_CARTERA,TIPO_FONDO,CARTERA,AGENCIA,ID_VTA
3,0011,2024-08-27,27204708,00248,195,186.0,PEN,ABRAHAM ALBERTO SEMINARIO SAN MARTIN,P,2,001101955001801884,ABRAHAM ALBERTO SEMINARIO SAN MARTIN,28-Ago,MIV,UNSECURED,NULL,KSTBC,MORNESE MORA,-
5,0011,2024-08-27,27204708,00248,195,1178.0,PEN,ABRAHAM ALBERTO SEMINARIO SAN MARTIN.,P,2,001101955001801884,ABRAHAM ALBERTO SEMINARIO SAN MARTIN,28-Ago,MIV,UNSECURED,NULL,KSTBC,MORNESE MORA,-
17,0011,2024-08-27,30434144,00248,900,1484.0,PEN,ALVARADO CASTRO FRESCIA,P,2,001108149603038528,FRESCIA LISET ALVARADO CASTRO,28-Ago,MIV,UNSECURED,NULL,EXJ,MORNESE MORA,-
19,0011,2024-08-27,30434144,00248,900,2166.0,PEN,ALVARADO CASTRO FRESCIA.,P,2,001108149603038528,FRESCIA LISET ALVARADO CASTRO,28-Ago,MIV,UNSECURED,NULL,EXJ,MORNESE MORA,-
22,0011,2024-08-27,22918561,00248,270,300.0,PEN,ANA LUZ VIVAS AGURTO TC,P,2,001102709600217948,ANA LUZ VIVAS AGURTO,28-Ago,MIV,UNSECURED,NULL,EXJ,ASESCOM RJ,-


In [131]:
multi_count = df_multi.shape[0]
multi_count

90

In [132]:
df_multi['TIPO_CARTERA'].value_counts()

TIPO_CARTERA
UNSECURED    78
SECURED      12
Name: count, dtype: int64

In [133]:
df_multi['TIPO_FONDO'].value_counts()

TIPO_FONDO
NULL        82
REACTIVA     8
Name: count, dtype: int64

In [134]:
df_no_multi = df_multi[(df_multi['TIPO_CARTERA'] != 'UNSECURED')]
no_multi_count = df_no_multi.shape[0]
no_multi_count

12

In [135]:
df_multi = df_multi[df_multi['TIPO_CARTERA'] == 'UNSECURED']

df_multi['CONTRATO'] = None
df_multi['TIPO_FONDO'] = None
df_multi['CARTERA'] = None

df_multi = df_multi.drop(columns=['TIPO_CARTERA', 'AGENCIA'])
df_multi = df_multi.drop_duplicates(subset=['CC', 'IMPORTE', 'MONEDA', 'NOMBRE'])

cols_multi = ['FECHA', 'CC', 'CLAVSERV', 'IMPORTE', 'MONEDA', 'NOMBRE', 'ESTADO', 'FLAG', 'CONTRATO', 'TIPO_FONDO', 'CARTERA', 'NOMBRE_CLIENTE', 'FECHA_ENVIO', 'ID_RESPONSABLE']
df_multi = df_multi[cols_multi]

print(df_multi.shape)
df_multi.head(5)

(78, 14)


,FECHA,CC,CLAVSERV,IMPORTE,MONEDA,NOMBRE,ESTADO,FLAG,CONTRATO,TIPO_FONDO,CARTERA,NOMBRE_CLIENTE,FECHA_ENVIO,ID_RESPONSABLE
3,2024-08-27,27204708,00248,186.0,PEN,ABRAHAM ALBERTO SEMINARIO SAN MARTIN,P,2,None,None,None,ABRAHAM ALBERTO SEMINARIO SAN MARTIN,28-Ago,MIV
5,2024-08-27,27204708,00248,1178.0,PEN,ABRAHAM ALBERTO SEMINARIO SAN MARTIN.,P,2,None,None,None,ABRAHAM ALBERTO SEMINARIO SAN MARTIN,28-Ago,MIV
17,2024-08-27,30434144,00248,1484.0,PEN,ALVARADO CASTRO FRESCIA,P,2,None,None,None,FRESCIA LISET ALVARADO CASTRO,28-Ago,MIV
19,2024-08-27,30434144,00248,2166.0,PEN,ALVARADO CASTRO FRESCIA.,P,2,None,None,None,FRESCIA LISET ALVARADO CASTRO,28-Ago,MIV
22,2024-08-27,22918561,00248,300.0,PEN,ANA LUZ VIVAS AGURTO TC,P,2,None,None,None,ANA LUZ VIVAS AGURTO,28-Ago,MIV


In [136]:
df_multi.to_excel(multiproducto, index=False)

In [137]:
multi_count_final = df_multi.shape[0]
multi_count_final

78

In [138]:
if multi_count == multi_count_final + no_multi_count:
    print('MULTIPRODUCTO OK')
else:
    print('MULTIPRODUCTO ERROR')

MULTIPRODUCTO OK


In [139]:
#################### FIN MULTIPRODUCTO ####################

In [140]:
#################### NO ENVIADOS ####################

In [141]:
no_eviados = no_mono_count + no_multi_count + no_flag_count
enviados = mono_count_final + multi_count_final + reactiva_count

print('No enviados:', no_eviados)
print('Enviados:', enviados)
print('Total:', no_eviados + enviados)

No enviados: 48
Enviados: 387
Total: 435


In [142]:
if base_count == enviados + no_eviados:
    print('REGISTROS OK')
else:
    print('REGISTROS ERROR')

REGISTROS OK


In [143]:
cols_no_enviados = ['ENTIDAD', 'FECHA', 'CC', 'CLAVSERV', 'CENTROPAGO', 'IMPORTE', 'MONEDA', 'NOMBRE', 'ESTADO', 'FLAG', 'TIPO_FONDO', 'TIPO_CARTERA', 'CONTRATO', 'ID_VTA']

In [144]:
df_no_mono = df_no_mono[cols_no_enviados]
df_no_multi = df_no_multi[cols_no_enviados]
df_base_no_flag = df_base_no_flag[cols_no_enviados]

print(df_no_mono.columns)
print(df_no_multi.columns)
print(df_base_no_flag.columns)

Index(['ENTIDAD', 'FECHA', 'CC', 'CLAVSERV', 'CENTROPAGO', 'IMPORTE', 'MONEDA',
       'NOMBRE', 'ESTADO', 'FLAG', 'TIPO_FONDO', 'TIPO_CARTERA', 'CONTRATO',
       'ID_VTA'],
      dtype='object')
Index(['ENTIDAD', 'FECHA', 'CC', 'CLAVSERV', 'CENTROPAGO', 'IMPORTE', 'MONEDA',
       'NOMBRE', 'ESTADO', 'FLAG', 'TIPO_FONDO', 'TIPO_CARTERA', 'CONTRATO',
       'ID_VTA'],
      dtype='object')
Index(['ENTIDAD', 'FECHA', 'CC', 'CLAVSERV', 'CENTROPAGO', 'IMPORTE', 'MONEDA',
       'NOMBRE', 'ESTADO', 'FLAG', 'TIPO_FONDO', 'TIPO_CARTERA', 'CONTRATO',
       'ID_VTA'],
      dtype='object')


In [145]:
df_no_enviados = pd.concat([df_no_mono, df_no_multi, df_base_no_flag])
df_no_enviados['CONTRATO'] = None
print(df_no_enviados.shape)
print(df_no_enviados.dtypes)
df_no_enviados.head(5)

(48, 14)
ENTIDAD          object
FECHA            object
CC               object
CLAVSERV         object
CENTROPAGO        int64
IMPORTE         float64
MONEDA           object
NOMBRE           object
ESTADO           object
FLAG              Int64
TIPO_FONDO       object
TIPO_CARTERA     object
CONTRATO         object
ID_VTA           object
dtype: object


,ENTIDAD,FECHA,CC,CLAVSERV,CENTROPAGO,IMPORTE,MONEDA,NOMBRE,ESTADO,FLAG,TIPO_FONDO,TIPO_CARTERA,CONTRATO,ID_VTA
0,0011,2024-08-27,23307614,00248,849,500.00,PEN,-VICTOR RAUL VELASQUEZ CENTENO,P,1,NULL,NULL,None,-
21,0011,2024-08-27,28123703,00248,156,20000.00,PEN,AMITEL CORP SAC .,P,1,NULL,SECURED,None,-
66,0011,2024-08-27,25994829,00248,361,2000.00,PEN,CASA FLAVIA SRL,P,1,REACTIVA,SECURED,None,-
88,0011,2024-08-27,27335710,00248,133,32781.55,PEN,CORPORACION DURANSA SAC ...,P,1,NULL,SECURED,None,-
199,0011,2024-08-27,30154284,00248,235,25000.00,PEN,INVERS. INMOV. CONSTRUCTORA DEL CENTRO,P,1,NULL,SECURED,None,-


In [146]:
if no_eviados == df_no_enviados.shape[0]:
    print('NO ENVIADOS OK')
else:
    print('NO ENVIADOS ERROR')

NO ENVIADOS OK


In [147]:
df_no_enviados.to_excel(no_enviados_path, index=False)

In [148]:
#################### FORMATO EXCEL ####################

In [149]:
import openpyxl as op
from openpyxl.styles import Font, PatternFill, Alignment

def format_excel(file_path: str, validator: str) -> None:
    workbook = op.load_workbook(file_path)
    sheet = workbook.active
    
    # Definir estilos generales
    general_font = Font(name='Calibri', size=11)
    header_font = Font(name='Calibri', size=11, bold=True, color='FFFFFF')
    alignment_center = Alignment(horizontal='center', vertical='center')
    
    # Definir estilos específicos
    header_font_white = Font(name='Calibri', size=11, bold=True, color='000000')
    header_fill_blue = PatternFill(start_color='002060', end_color='002060', fill_type='solid')
    header_fill_yellow = PatternFill(start_color='FFD965', end_color='FFD965', fill_type='solid')
    header_fill_green = PatternFill(start_color='C4D79B', end_color='C4D79B', fill_type='solid')
    header_fill_orange = PatternFill(start_color='FABF8F', end_color='FABF8F', fill_type='solid')
    
    # Aplicar estilos generales a todas las celdas
    for row in sheet.iter_rows():
        for cell in row:
            cell.font = general_font
    
    # Aplicar estilos al encabezado (fila 1)
    for cell in sheet[1]:
        cell.font = header_font
        cell.alignment = alignment_center
    
    # Aplicar estilos específicos según el validador
    if validator == 'mono':
        for col in range(1, 10):  # Columnas A-I
            sheet.cell(row=1, column=col).fill = header_fill_blue
        for col in range(10, 14):  # Columnas J-M
            sheet.cell(row=1, column=col).fill = header_fill_yellow
            sheet.cell(row=1, column=col).font = header_font_white
        for col in range(14, 16):  # Columnas N-O
            sheet.cell(row=1, column=col).fill = header_fill_green
            sheet.cell(row=1, column=col).font = header_font_white
        for col in range(16, 17):  # Columna P
            sheet.cell(row=1, column=col).fill = header_fill_orange
            sheet.cell(row=1, column=col).font = header_font_white
    
    elif validator == 'multi':
        for col in range(1, 10):  # Columnas A-I
            sheet.cell(row=1, column=col).fill = header_fill_blue
        for col in range(10, 12):  # Columnas J-K
            sheet.cell(row=1, column=col).fill = header_fill_green
            sheet.cell(row=1, column=col).font = header_font_white
        for col in range(12, 15):  # Columnas L-N
            sheet.cell(row=1, column=col).fill = header_fill_yellow
            sheet.cell(row=1, column=col).font = header_font_white
    
    elif validator == 'react':
        for col in range(1, 10):  # Columnas A-I
            sheet.cell(row=1, column=col).fill = header_fill_blue
        for col in range(10, 14):  # Columnas J-M
            sheet.cell(row=1, column=col).fill = header_fill_yellow
            sheet.cell(row=1, column=col).font = header_font_white
        for col in range(14, 16):  # Columnas N-O
            sheet.cell(row=1, column=col).fill = header_fill_green
            sheet.cell(row=1, column=col).font = header_font_white
        for col in range(16, 17):  # Columna P
            sheet.cell(row=1, column=col).fill = header_fill_orange
            sheet.cell(row=1, column=col).font = header_font_white
    
    elif validator == 'no_env':
        for col in range(1, 11): # Columnas A-J
            sheet.cell(row=1, column=col).fill = header_fill_blue
        for col in range(11, 15): # Columnas K-N
            sheet.cell(row=1, column=col).fill = header_fill_yellow
            sheet.cell(row=1, column=col).font = header_font_white
    
    workbook.save(file_path)

In [150]:
format_excel(mono_path, "mono")
format_excel(multi_path, "multi")
format_excel(react_path, "react")
format_excel(no_enviados_path, "no_env")

os.startfile(mono_path)
os.startfile(multi_path)
os.startfile(react_path)
os.startfile(no_enviados_path)